# HW05: Double ML with XGBoost
(due October 27th)

In [10]:
# Load Data (NLSY)
import pandas as pd
df = pd.read_stata('http://www.stata-press.com/data/r10/nlswork.dta')
df = df.dropna()

In [11]:
import numpy as np
Y = df['ln_wage']
D = df['union']

In [12]:
df.head()

,idcode,year,birth_yr,age,race,msp,nev_mar,grade,collgrad,not_smsa,...,south,ind_code,occ_code,union,wks_ue,ttl_exp,tenure,hours,wks_work,ln_wage
2,1,72,51,20.0,2,1.0,0.0,12.0,0,0.0,...,0.0,4.0,6.0,1.0,0.0,2.256410,0.916667,40.0,51.0,1.589977
5,1,77,51,25.0,2,0.0,0.0,12.0,0,0.0,...,0.0,12.0,8.0,0.0,0.0,3.775641,1.500000,32.0,52.0,1.778681
7,1,80,51,28.0,2,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,5.294872,1.833333,45.0,75.0,2.551715
9,1,85,51,33.0,2,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,7.160256,1.916667,42.0,97.0,2.614172
10,1,87,51,35.0,2,0.0,0.0,12.0,0,0.0,...,0.0,5.0,6.0,1.0,0.0,8.987180,3.916667,45.0,95.0,2.536374


In [13]:
# fill in X with all predictors that are not colliders
X = df[['age', 'year', 'race', 'msp','collgrad', 'nev_mar', 'grade', 'not_smsa', 'south' ]]

In [37]:
from sklearn.model_selection import train_test_split

# split into sample A and sample B
X_a, X_b, D_a, D_b, y_a, y_b = train_test_split(X, D, Y, test_size=0.5, random_state=42)

# Within each sample, make a validation set for xgboost early stopping
X_a_val_train, X_a_test, D_a_val_train, D_a_test, y_a_val_train, y_a_test = train_test_split(X_a, D_a, y_a, test_size=0.3, random_state=42)
X_a_train, X_a_val, D_a_train, D_a_val, y_a_train, y_a_val = train_test_split(X_a_val_train, D_a_val_train, y_a_val_train, test_size=0.2, random_state=42)

X_b_val_train, X_b_test, D_b_val_train, D_b_test, y_b_val_train, y_b_test = train_test_split(X_b, D_b, y_b, test_size=0.3, random_state=42)
X_b_train, X_b_val, D_b_train, D_b_val, y_b_train, y_b_val = train_test_split(X_b_val_train, D_b_val_train, y_b_val_train, test_size=0.2, random_state=42)

In [38]:
# Step 1. In both samples, use xgboost regressor to predict log wages (outcome Y)
# use early stopping.
from sklearn.metrics import (mean_squared_error, roc_auc_score, f1_score)
from xgboost import XGBRegressor
xgbr_a = XGBRegressor()
xgbr_a.fit(X_a_train, y_a_train, early_stopping_rounds=5, eval_set=[(X_a_val, y_a_val)])
y_hat_a = xgbr_a.predict(X_a_test)

xgbr_b = XGBRegressor()
xgbr_b.fit(X_b_train, y_b_train, early_stopping_rounds=5, eval_set = [(X_b_val, y_b_val)])
y_hat_b = xgbr_b.predict(X_b_test)
print()
print('RMSE on Test Set')
print('Sample A: %f' % np.sqrt(mean_squared_error(y_hat_a, y_a_test)))
print('Sample B: %f' % np.sqrt(mean_squared_error(y_hat_b, y_b_test)))

[0]	validation_0-rmse:0.96856
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.73962
[2]	validation_0-rmse:0.59363
[3]	validation_0-rmse:0.50579
[4]	validation_0-rmse:0.45773
[5]	validation_0-rmse:0.43157
[6]	validation_0-rmse:0.41793
[7]	validation_0-rmse:0.41093
[8]	validation_0-rmse:0.40872
[9]	validation_0-rmse:0.40771
[10]	validation_0-rmse:0.40701
[11]	validation_0-rmse:0.40685
[12]	validation_0-rmse:0.40695
[13]	validation_0-rmse:0.40781
[14]	validation_0-rmse:0.40704
[15]	validation_0-rmse:0.40797
[16]	validation_0-rmse:0.40875
Stopping. Best iteration:
[11]	validation_0-rmse:0.40685

[0]	validation_0-rmse:0.93364
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.70897
[2]	validation_0-rmse:0.56635
[3]	validation_0-rmse:0.47999
[4]	validation_0-rmse:0.43155
[5]	validation_0-rmse:0.40466
[6]	validation_0-rmse:0.39092
[7]	validation_0-rmse:0.38383
[8]	validation_0-rmse:0.38026
[9]	validation_0-rmse:0.378

In [39]:
# Step 2. In both samples, use xgboost classifier to predict union status (treatment D)
# use early stopping.
from xgboost import XGBClassifier
xgbc_a = XGBClassifier()
xgbc_a.fit(X_a_train, D_a_train, early_stopping_rounds=5, eval_set=[(X_a_val, D_a_val)])
D_hat_a = xgbc_a.predict(X_a_test)

xgbc_b = XGBClassifier()
xgbc_b.fit(X_b_train, D_b_train, early_stopping_rounds=5, eval_set = [(X_b_val, D_b_val)])
D_hat_b = xgbc_b.predict(X_b_test)
print()
print('F1 Score on Test Set')
print('Sample A: %f' % accuracy_score(D_hat_a, D_a_test))
print('Sample B: %f' % accuracy_score(D_hat_b, D_b_test))

[0]	validation_0-error:0.23885
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.24628
[2]	validation_0-error:0.24628
[3]	validation_0-error:0.25372
[4]	validation_0-error:0.24098
[5]	validation_0-error:0.24416
Stopping. Best iteration:
[0]	validation_0-error:0.23885

[0]	validation_0-error:0.23036
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.22824
[2]	validation_0-error:0.22187
[3]	validation_0-error:0.22293
[4]	validation_0-error:0.22187
[5]	validation_0-error:0.22081
[6]	validation_0-error:0.22399
[7]	validation_0-error:0.22505
[8]	validation_0-error:0.22612
[9]	validation_0-error:0.22505
[10]	validation_0-error:0.22399
Stopping. Best iteration:
[5]	validation_0-error:0.22081


F1 Score on Test Set
Sample A: 0.766105
Sample B: 0.780476


In [40]:
# Step 3. Cross-fitting: Form predictions in other sample.

# predict wages in sample A using model trained in sample B:
y_hat_A = xgbr_b.predict(X_a_test)

# vice versa:
y_hat_B = xgbr_a.predict(X_b_test)

print('RMSE on Test Set')
print('Sample A: %f' % np.sqrt(mean_squared_error(y_hat_A, y_a_test)))
print('Sample B: %f' % np.sqrt(mean_squared_error(y_hat_B, y_b_test)))
    
# predict union status in sample A using model trained in sample B:
D_hat_A = xgbc_b.predict(X_a_test)
    
# vice versa:
D_hat_B = xgbc_a.predict(X_b_test)

print()
print('F1 Score on Test Set')
print('Sample A: %f' % accuracy_score(D_hat_A, D_a_test))
print('Sample B: %f' % accuracy_score(D_hat_B, D_b_test))

RMSE on Test Set
Sample A: 0.394763
Sample B: 0.396818

F1 Score on Test Set
Sample A: 0.769574
Sample B: 0.775520


In [44]:
# Compute residuals for wages and union status.

# residualized wages in samples A and B:
y_tilde_A =  y_a_test - y_hat_A
y_tilde_B =  y_b_test - y_hat_B

# residualized union status in samples A and B:
D_tilde_A =  D_a_test - D_hat_A
D_tilde_B =  D_b_test - D_hat_B

In [45]:
# Step 4. Run OLS regressions and produce DML estimate

import statsmodels.api as sm

model_A = sm.OLS(y_tilde_A, D_tilde_A)
result_A = model_A.fit(cov_type='HC1')
beta_A = result_A.params
se_A = result_A.bse

model_B = sm.OLS(y_tilde_B, D_tilde_B)
result_B = model_B.fit(cov_type='HC1')
beta_B = result_B.params
se_B = result_B.bse

beta = .5 * (beta_A + beta_B)
se = .5 * (se_A + se_B)

print('DML Coeff:',beta)
print('DML S.E.:', se)

DML Coeff: union    0.160911
dtype: float64
DML S.E.: union    0.017513
dtype: float64
